# ESG profile Key-Initiatives (KIs) from Companies' Annual reports
> * Extract top global Companies list (from Report evaluated by PwC with Bloomberg)
> * Extract KIs from Companys' Annual reports

In [1]:
# Import neccessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Extraction libraries

#! pip install PyPDF2
import PyPDF2
import re

In [3]:
# Extract PDF data

pdf_file = open('D:\GitHub_Projects\esg-profile\global-top-100-companies-june-2020-update.pdf', 'rb')
pdf_data = PyPDF2.PdfFileReader(pdf_file)
print("Total Pages of Report:", pdf_data.numPages)

Total Pages of Report: 17


In [4]:
# Extract tables from pdf_data
pages = [10, 11, 12, 13, 14]
table_data = []

for page in pages:
    table = pdf_data.getPage(page).extractText()
    table_data.append(table)

print(table_data)

['PwC \n| Global Top 100 companies by market capitalisation\n11Top 100 global companies 1\n-20\nSource: Bloomberg with PwC analysis\n30 June 2020\n31 March 2020\n31 Dec 2019\nRank\nCompany name\nLocation\nSector\nRank +/\n-(VS Mar 2020)\nRank +/\n-(VS Dec 2019)\nMarket  \ncapitalisation \n\n($bn)\nRank\nMarket \n\ncapitalisation \n\n($bn)\nRank\nMarket \ncapitalisation \n($bn)\n1SAUDI ARABIAN OI \nSaudi Arabia \nOil & Gas \n001,741 \n1 1,602 \n1 1,879 \n2APPLE INC \nUnited States \nTechnology \n101,568 \n3 1,113 \n2 1,305 \n3MICROSOFT CORP \nUnited States \nTechnology \n-101,505 \n2 1,200 \n3 1,203 \n4AMAZON.COM INC \nUnited States \nConsumer Services \n011,337 \n4 971 5 916 5ALPHABET INC\n-A United States \nTechnology \n0-1953 5 799 4 923 6FACEBOOK INC\n-A United States \nTechnology \n10629 7 475 6 585 7TENCENT\nMainland China \nTechnology \n12599 8 469 9 461 8ALIBABA GRP\n-ADR \nMainland China \nConsumer Services \n-2-1577 6 522 7 569 9BERKSHIRE HATH\n-A United States \nFinancials \n

In [24]:
# Filter for data from columns - 'Rank', 'Company_Name', 'Country', 'Sector'

ranks = list(range(1, 101))
company_data = []

for table in list(range(0,5)):
    regex = re.findall('[A-Z\s\W?a-z]+', table_data[table])
    filter_data = [ele for ele in regex if ele != ' ' and ' \n' and ',' and '-']
    company_data.append(filter_data)

In [25]:
company_data

[['PwC \n| Global Top ',
  ' companies by market capitalisation\n',
  'Top ',
  ' global companies ',
  '\n-',
  '\nSource: Bloomberg with PwC analysis\n',
  ' June ',
  '\n',
  ' March ',
  '\n',
  ' Dec ',
  '\nRank\nCompany name\nLocation\nSector\nRank +/\n-(VS Mar ',
  ')\nRank +/\n-(VS Dec ',
  ')\nMarket  \ncapitalisation \n\n($bn)\nRank\nMarket \n\ncapitalisation \n\n($bn)\nRank\nMarket \ncapitalisation \n($bn)\n',
  'SAUDI ARABIAN OI \nSaudi Arabia \nOil & Gas \n',
  ',',
  ' \n',
  ',',
  ' \n',
  ',',
  ' \n',
  'APPLE INC \nUnited States \nTechnology \n',
  ',',
  ' \n',
  ',',
  ' \n',
  ',',
  ' \n',
  'MICROSOFT CORP \nUnited States \nTechnology \n-',
  ',',
  ' \n',
  ',',
  ' \n',
  ',',
  ' \n',
  'AMAZON.COM INC \nUnited States \nConsumer Services \n',
  ',',
  ' \n',
  'ALPHABET INC\n-A United States \nTechnology \n',
  '-',
  'FACEBOOK INC\n-A United States \nTechnology \n',
  'TENCENT\nMainland China \nTechnology \n',
  'ALIBABA GRP\n-ADR \nMainland China \nConsume

In [28]:
regex_country = re.findall('[A-Z\s\W?a-z]+', table_data[0])
regex_country = [row for row in regex_country if row != ' ' and ' \n' and ',' and '-']
#regex_country[len(regex_country)-20 : ]
regex_country

['PwC \n| Global Top ',
 ' companies by market capitalisation\n',
 'Top ',
 ' global companies ',
 '\n-',
 '\nSource: Bloomberg with PwC analysis\n',
 ' June ',
 '\n',
 ' March ',
 '\n',
 ' Dec ',
 '\nRank\nCompany name\nLocation\nSector\nRank +/\n-(VS Mar ',
 ')\nRank +/\n-(VS Dec ',
 ')\nMarket  \ncapitalisation \n\n($bn)\nRank\nMarket \n\ncapitalisation \n\n($bn)\nRank\nMarket \ncapitalisation \n($bn)\n',
 'SAUDI ARABIAN OI \nSaudi Arabia \nOil & Gas \n',
 ',',
 ' \n',
 ',',
 ' \n',
 ',',
 ' \n',
 'APPLE INC \nUnited States \nTechnology \n',
 ',',
 ' \n',
 ',',
 ' \n',
 ',',
 ' \n',
 'MICROSOFT CORP \nUnited States \nTechnology \n-',
 ',',
 ' \n',
 ',',
 ' \n',
 ',',
 ' \n',
 'AMAZON.COM INC \nUnited States \nConsumer Services \n',
 ',',
 ' \n',
 'ALPHABET INC\n-A United States \nTechnology \n',
 '-',
 'FACEBOOK INC\n-A United States \nTechnology \n',
 'TENCENT\nMainland China \nTechnology \n',
 'ALIBABA GRP\n-ADR \nMainland China \nConsumer Services \n-',
 '-',
 'BERKSHIRE HATH\n-A

### Alternate approach
> * use table from 'https://en.wikipedia.org/wiki/List_of_largest_companies_by_revenue'

In [ ]:
pd.read_html('https://en.wikipedia.org/wiki/List_of_largest_companies_by_revenue')[0].head()

In [ ]:
# Table Data extraction, if HTML page was extracted

data = []
table = raw.find('table', attrs={'class':'fbs-table organization'})
table_body = table.find('tbody')
table_rows = table_body.findAll('tr')

for row in table_rows:
    cols = row.find_all('td')
    cols = [element.text.strip() for element in row]
    data.append([element for element in cols if element])

data


In [ ]:
# Group list of companies by sectors (https://en.wikipedia.org/wiki/List_of_companies_of_Finland)

companies_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_companies_of_Finland')[0]
companies_df.head()

In [ ]:
columns = companies_df.columns.to_list()
columns

In [ ]:
# changing wrong column label, Sector --> Company

companies_df.columns = ['Company_Name', 'Industry', 'Company_Type', 'Headquarters', 'Founded', 'Notes']
companies_df.info()

## Exploratory Data Analysis

In [ ]:
# Calculate Age of Companies in years w.r.t completed year (2020)

companies_df['Company_Age'] = companies_df['Founded'].apply(lambda yr: 2020 - yr)
companies_df.head()

In [ ]:
# Companies greater than & less than or equal to 100 years
age_g100 = companies_df.loc[companies_df['Company_Age']>100].Company_Name.to_list()
age_le100 = companies_df.loc[companies_df['Company_Age']<=100].Company_Name.to_list()
print(f"Companies older than 100 years:\n{age_g100}\n\nCompanies younger than 100 years:\n{age_le100}")

In [ ]:
#! pip install google

In [ ]:
def company_reports(companies):
    """
    Provides a list of links related to companies' sustainability reports, if available

    Args:
    companies - list of companies
    
    Returns:
    company_links - list of websites related to companies' list provided
    """

    from googlesearch import search
    
    company_links = []

    for company in companies:
        query = f"annual sustainability report of {company}"
        try:
            for link in search(query, lang='fi', tld='.fi', country='finland', num=10, start=1, stop=10, pause=2.5):
                company_links.append(link)
        except UnicodeError:
            print(f"Excluding {company} due to name error")

    return company_links

In [ ]:
g100_links = company_reports(age_g100)
le100_links = company_reports(age_le100)

print(g100_links, '\n', le100_links)

In [ ]:
# Companies visualized w.r.t. type of Industries

companies_df.groupby('Industry')['Company_Type'].count().plot(kind='barh', figsize=(20,10), title='Company types under different Industries')
#sns.catplot(x='Sector', hue='Industry', col='Founded', data=companies_df, kind='count', height=7, aspect=0.8)

In [ ]:
# Company_Types visualized specifically w.r.t. types of Industry

plt.figure(figsize=(20,10))
sns.countplot(y='Company', hue='Industry', data=companies_df)